In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup as bsoup
import requests
import pandas as pd
print('Packages fully loaded and installed.')

Packages fully loaded and installed.


In [98]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= bsoup(source, 'lxml')
#print(soup.prettify())
table= soup.find('table',class_ = 'wikitable')
tabledata = table.tbody.text.split('\n\n')
tableitems = []

for rows in tabledata:
    temp  = rows.split('\n')[1:]
    if (temp != []):
       if (temp[1]!='Not assigned'):
        if(temp[2] == 'Not assigned'):
            temp[2] = temp[1]
        
        tableitems.append(temp)
tableitems.sort()

i = 0
garbage = pd.DataFrame()
for item in tableitems:
    if (i+1< len(tableitems) and tableitems[i][0]==tableitems[i+1][0]):
         print(tableitems[i][2],tableitems[i+1][2] )
        #concatenate the towns while this is true
    else:
        print('f')
        #add to DF as is
    i = i+1
    

Malvern Rouge
f
Highland Creek Port Union
Port Union Rouge Hill
f
Guildwood Morningside
Morningside West Hill
f
f
f
f
East Birchmount Park Ionview
Ionview Kennedy Park
f
Clairlea Golden Mile
Golden Mile Oakridge
f
Cliffcrest Cliffside
Cliffside Scarborough Village West
f
Birch Cliff Cliffside West
f
Dorset Park Scarborough Town Centre
Scarborough Town Centre Wexford Heights
f
Maryvale Wexford
f
f
Clarks Corners Sullivan
Sullivan Tam O'Shanter
f
Agincourt North L'Amoreaux East
L'Amoreaux East Milliken
Milliken Steeles East
f
f
f
f
Fairview Henry Farm
Henry Farm Oriole
f
f
Silver Hills York Mills
f
Newtonbrook Willowdale
f
f
f
f
f
f
Don Mills South Flemingdon Park
f
Bathurst Manor Downsview North
Downsview North Wilson Heights
f
Northwood Park York University
f
CFB Toronto Downsview East
f
f
f
f
f
Parkview Hill Woodbine Gardens
f
f
f
f
f
f
Riverdale The Danforth West
f
India Bazaar The Beaches West
f
f
f
f
f
f
Moore Park Summerhill East
f
Deer Park Forest Hill SE
Forest Hill SE Rathnelly

In [62]:
toronto_df = pd.DataFrame(tableitems)

new_header = toronto_df.iloc[0]
toronto_df = toronto_df[1:]
toronto_df.columns = new_header
toronto_df.head()

,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Rouge Hill
5,M1E,Scarborough,Guildwood
